In [1]:
# Import needed libraries
import warnings
import subprocess
from io import BytesIO

import folium
from folium import plugins
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv

warnings.filterwarnings("ignore")

## Analyzing the annual maximum streamflow of the San Gregorio Creek reveals a notable correlation between heavy rain years and increased landslides in the region.

#### Location of San Gregorio Creek stream guage:

In [2]:
# Location map of stream gauge

# Variables
sg_lat = 37.3257762
sg_lon = -122.3866404

# Initialize map and tweak settings
fig = folium.Figure(width=1000, height=500)
map = folium.Map(
    location=(sg_lat, sg_lon),
    tiles="openstreetmap",
    zoom_start=14,
    scrollWheelZoom=False,).add_to(fig)

minimap = plugins.MiniMap(zoom_level_offset=-6.5, width=200, height=200)
map.add_child(minimap)


# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Stream Guage Location").add_to(map)

# Display the map
map

In [3]:
# Load the NWIS San Gregorio Creek streamflow data
site_url = ('https://waterdata.usgs.gov/nwis/dv?'
             'cb_00060=on&format=rdb'
             '&site_no=11162570'
             '&legacy='
             '&referred_module=sw'
             '&period='
             '&begin_date=1969-10-01&end_date=2023-09-17')

# Send an HTTP GET request to the URL
sgre_data = requests.get(site_url)

# Check if the request was successful (status code 200)
#if sgre_data.status_code == 200:
 #   # Print the content of the response (HTML, JSON, etc.)
  #  print(sgre_data.text)
#else:
 #   print(f"Request failed with status code {sgre_data.status_code}")



In [4]:
# Print the top of the data
#for i, line in enumerate(sgre_data.content.splitlines()[:10]):
 #   print(i, line)

In [5]:
# Take a look at the data. What got downloaded?
#for i, line in enumerate(sgre_data.content.splitlines()[:35]):
 #   # Skip commented lines
  #  if not line.startswith(b'#'):
   #     print(i, line)

In [6]:
# Clean the data for use
sgre_data_df = pd.read_csv(
    BytesIO(sgre_data.content),
    comment='#',
    delimiter='\t',
    skiprows=[29, 30],
    names=['Agency', 'site_no', 'datetime', 'streamflow_cfs', 'code'],
    index_col='datetime',
    parse_dates=True)


In [7]:
# Get data type info
#sgre_data_df.info()

In [8]:
# Plot all of the streamflow data
#sgre_data_df.streamflow_cfs.hvplot(
#   title='Daily Streamflow of the San Gregorio Creek',
#     xlabel='Date',
#     ylabel='Mean discharge (cfs)')

In [9]:
# Pull out the maximum yearly streamflow
# The 'AS' means Year start frequency (i.e. start at Jan. 1)
sgre_ann_max_df = sgre_data_df[["streamflow_cfs"]].resample("AS").max()


In [10]:
# I noticed a bunch of NaN data so I want to filter out the years of no data
filter_df = sgre_ann_max_df[sgre_ann_max_df['streamflow_cfs'].notna()]


In [11]:
# Extract the year from the exisiting year/month data
filter_df['year'] = filter_df.index.year


In [12]:
# Plot the subset of data
#filter_df.hvplot(x='year', y='streamflow_cfs',
 #   title='Annual Maximum Streamflow of the San Gregorio Creek',
  #    xlabel='Year',
   #   ylabel='Mean discharge (cfs)')



In [13]:
# Labeled plot
labels = [hv.Text(x=1973, y=2200, text='!!'),
          hv.Text(x=1982, y=4300, text='!!'),
          hv.Text(x=1986, y=2500, text='!!'),
          hv.Text(x=1998, y=1000, text='!!'),
          hv.Text(x=2017, y=1950, text='!!'),
          hv.Text(x=2022, y=3100, text='!!')]
plot_with_labels = filter_df.hvplot(x='year', y='streamflow_cfs', title='Annual Maximum Streamflow of the San Gregorio Creek', xlabel='Year', ylabel='Mean discharge (cfs)') * hv.Overlay(labels)
plot_with_labels

:Overlay
   .Curve.I  :Curve   [year]   (streamflow_cfs)
   .Text.I   :Text   [x,y]
   .Text.II  :Text   [x,y]
   .Text.III :Text   [x,y]
   .Text.IV  :Text   [x,y]
   .Text.V   :Text   [x,y]
   .Text.VI  :Text   [x,y]

The double exclamation marks signify the rain years when heightened landslide activity occurred.  It's important to note that the stream guage was non-operational during the period of 1995 - 2000.

Citation:

U.S. Geological Survey, 2016, National Water Information System data available on the World Wide Web (USGS Water Data for the Nation), accessed [September 18, 2023], at URL https://waterdata.usgs.gov/nwis/inventory/?site_no=11162570&agency_cd=USGS.

Rainfall and Landslides in Northern California | U.S. Geological Survey. 2 Apr. 2019, https://www.usgs.gov/programs/landslide-hazards/science/rainfall-and-landslides-northern-california .

In [14]:
%%capture
%%bash
jupyter nbconvert SanGregorioCreek_time-series.ipynb --to html --no-input
